# Build a RAG Agent in Langchain

Building a Retrieval-Augmented Generation (RAG) agent in Langchain involves several steps, including setting up the environment, loading and processing documents, creating a vector store, and defining the agent's behavior. Below is a step-by-step guide to building a RAG agent in Langchain.

# Setup Environment

In [ ]:
!pip install langchain langchain-text-splitters langchain-community bs4

In [ ]:
!pip install -U "langchain-openai"

In [1]:
import os
import getpass

# Set your OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [2]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
from langchain_core.vectorstores import InMemoryVectorStore

# Create an in-memory vector store
vector_store = InMemoryVectorStore(embeddings)

# Indexing Documents

Load and process documents to create a vector store that the RAG agent can query for relevant information during generation.

LOAD >> SPLIT >> STORE

## Load Documents

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Load documents from a web page
# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert(len(docs) > 0), "No documents were loaded from the web page."
print(f"Loaded {len(docs)} documents from the web page.")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Loaded 1 documents from the web page.


## Split Documents

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
split_docs = text_splitter.split_documents(docs)
print(f"Split into {len(split_docs)} chunks.")

Split into 63 chunks.


## Store in Vector Store

In [6]:
doc_ids = vector_store.add_documents(split_docs)
print(f"Added {len(split_docs)} chunks to the vector store.")
print("Document IDs:", doc_ids)

Added 63 chunks to the vector store.
Document IDs: ['4a95ac0b-0231-46fb-97b1-8090028c5eeb', '1e7cf92c-54e7-4406-8264-c3c84d8fe57c', 'eb1d727d-fb57-483a-8a75-9ee31944cc81', '98eac3c4-9df4-496e-bea9-40756f230fb4', 'e5df745b-5252-4549-8fe0-b32548440dab', '91ede074-9e2d-41ed-9d8a-9fc82e8e3a06', 'e7a562e6-b66e-4bcc-83ba-b20f347d2f85', '9a6ff857-e4e9-4d12-957d-f5d118437e21', '6219591c-59a0-4de9-9529-dd2b817ee7da', '261a9f1a-a06a-4f6c-96a0-57488c350958', '94f5e9c3-f8e4-4cbf-babf-c721844bc6ad', 'c4f051d8-3543-4c7d-9382-16c9f8f861a7', '4fd36297-9a58-48c5-8e13-3d9ccd865b11', 'dafb858d-e354-4354-8920-36625e83bd9f', '4ee7908a-95c5-4fbc-8837-58d43b8737fa', '6ebceb89-32b6-4ad0-852f-bffb057e4612', 'a0679b80-bce0-4887-903c-66ebf4338d85', '1454ef98-86ff-46c6-b2ce-17560ff9cc22', '51e44e25-7799-4e70-a7cd-cf9de880a4df', 'e0ca545b-3224-4c96-93e8-2477d43a57dc', '9b2bbe3a-a76a-4488-9314-1f1e6f131690', '9470a7c8-e0d7-41e5-b697-ad55dab3a597', '17cc92e3-0caf-4639-8d41-a41a2ad08b06', 'f3f166ee-6434-4d56-a844-54f

# RAG Agents

In [10]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def search_docs(query: str) -> str:
    """Search for relevant documents in the vector store."""
    results = vector_store.similarity_search(query, k=3)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in results
    )
    return serialized, results

In [11]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

model = init_chat_model(model="gpt-4o")

agent = create_agent(
    tools=[search_docs],
    model=model,
)

## Test the Agent

In [12]:
query = "What are the key components of a RAG agent?"

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What are the key components of a RAG agent?
================================== Ai Message ==================================
Tool Calls:
  search_docs (call_dOpAWpI8o1qRuvVSYYPlGtnN)
 Call ID: call_dOpAWpI8o1qRuvVSYYPlGtnN
  Args:
    query: key components of a RAG agent
================================= Tool Message =================================
Name: search_docs

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 24126}
Content: They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out of 11 requests (36%) were accepted to obtain a synthesis solution and the agent attempted to consult documentation to execute the procedure. 7 out of 11 were rejected and among these 7 rejected cases, 5 happened after a Web search while 2 we